In [1]:
import os
import torch
print("Using torch", torch.__version__)

Using torch 2.1.0+cu118


In [2]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.1.0+cu118.html
!pip install pyg-library

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html


In [3]:
from torch_geometric.data import Data
from torch_geometric.datasets import MovieLens100K
from torch_geometric import nn
import torch_geometric.transforms as T

In [4]:
dataset = MovieLens100K(root='/tmp/movielens')

In [5]:
movielens_raw = dataset[0]
movielens_raw

HeteroData(
  movie={ x=[1682, 18] },
  user={ x=[943, 24] },
  (user, rates, movie)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
    edge_label_index=[2, 20000],
    edge_label=[20000],
  },
  (movie, rated_by, user)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
  }
)

In [6]:
del movielens_raw[("user", "rates", "movie")].edge_label_index
del movielens_raw[("user", "rates", "movie")].edge_label

In [7]:
movielens_raw

HeteroData(
  movie={ x=[1682, 18] },
  user={ x=[943, 24] },
  (user, rates, movie)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
  },
  (movie, rated_by, user)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
  }
)

In [8]:
node_types, edge_types = movielens_raw.metadata()
print(node_types)
print(edge_types)

['movie', 'user']
[('user', 'rates', 'movie'), ('movie', 'rated_by', 'user')]


In [9]:
transform = T.Compose([
    T.RandomLinkSplit(
        num_val=0.05,
        num_test=0.1,
        disjoint_train_ratio=0.2,   # supervision
        add_negative_train_samples=False,
        neg_sampling_ratio=1.0,
        edge_types=("user", "rates", "movie"),
        rev_edge_types=('movie', 'rated_by', 'user')
    )
])

In [10]:
train_data, val_data, test_data = transform(movielens_raw)

In [11]:
def user_importance(num_users, edge_index):
    """
    Calculates and normalizes the importance score for each user based on the average number of edges they are involved in.

    Parameters:
    num_users (int): The number of users.
    edge_index (Tensor): 2D tensor of edge indices, where the first element of each pair is a user label.

    Returns:
    Tensor: 1D tensor of importance scores for each user.
    """
    # Create a range of user labels based on the number of users
    user_labels = torch.arange(num_users)

    # Count the edges for each user
    edge_counts = torch.zeros(num_users, dtype=torch.float32)
    for user in edge_index[0, :]:
        if user < num_users:
            edge_counts[user] += 1


    # Calculate the importance score for each user
    # Users closer to the average have a higher score
    importance_scores = 1 / (1 + torch.abs(edge_counts - torch.mean(edge_counts))/torch.std(edge_counts))


    return importance_scores

user_score=user_importance(movielens_raw["user"].num_nodes, movielens_raw[("user", "rates", "movie")].edge_index)

In [12]:
print(user_score)

tensor([0.6256, 0.6515, 0.5959, 0.5419, 0.9315, 0.7691, 0.3954, 0.6045, 0.5350,
        0.9014, 0.8061, 0.5834, 0.2253, 0.6566, 0.7714, 0.8411, 0.5600, 0.5305,
        0.5283, 0.5875, 0.8918, 0.8496, 0.9636, 0.6566, 0.6566, 0.8165, 0.5283,
        0.6464, 0.5527, 0.5834, 0.5754, 0.5715, 0.5419, 0.5283, 0.5527, 0.5250,
        0.6366, 0.7933, 0.5385, 0.5715, 0.6089, 0.9636, 0.7552, 0.9454, 0.6002,
        0.5316, 0.5490, 0.6178, 0.7908, 0.5385, 0.5490, 0.6271, 0.5490, 0.6045,
        0.5283, 0.9212, 0.7310, 0.9454, 0.3917, 0.7104, 0.5350, 0.6871, 0.6947,
        0.7762, 0.6947, 0.5676, 0.5350, 0.5563, 0.6415, 0.9145, 0.5715, 0.8763,
        0.6224, 0.5676, 0.6724, 0.6778, 0.6464, 0.5419, 0.5917, 0.5419, 0.6045,
        0.9863, 0.9246, 0.6670, 0.5305, 0.5316, 0.7485, 0.5316, 0.6515, 0.5142,
        0.7064, 0.4129, 0.5250, 0.3810, 0.5373, 0.5959, 0.6271, 0.5490, 0.7933,
        0.6224, 0.6415, 0.7552, 0.5455, 0.7933, 0.5490, 0.5959, 0.5385, 0.5527,
        0.6303, 0.9673, 0.5350, 0.5794, 

In [13]:
def edge_importance(user_importance, edge_index,labels):
    """
    Calculates the importance score for each edge based on the user importance scores.

    Parameters:
    user_importance_scores (Tensor): 1D tensor of user importance scores.
    edge_index (Tensor): 2D tensor of edge indices with shape [2, num_edges].

    Returns:
    Tensor: 1D tensor of importance scores for each edge.
    """
    # Number of edges is the size of the second dimension of edge_index
    num_edges = edge_index.size(1)

    # Initialize a tensor to hold the importance scores for each edge
    edge_scores = torch.zeros(num_edges, dtype=torch.float32)

    # Assign each edge the importance score of the user it involves
    for i in range(num_edges):
        if(labels[i]==1):
            user = edge_index[0, i]
            edge_scores[i] = user_importance[user]
        else:
            edge_scores[i] = user_importance.max()



    return edge_scores

In [14]:
import sys
import torch_geometric
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, GINConv
from torch_geometric.nn import BatchNorm, LayerNorm, HeteroBatchNorm, HeteroLayerNorm
from torch_geometric.nn import to_hetero

class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()

        self.relu = torch.nn.ReLU()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, out_channels)

    def forward(self, node_feature, edge_index):

        output1 = self.relu(self.conv1(node_feature, edge_index))
        output2 = self.relu(self.conv2(output1, edge_index))
        output = self.conv3(output2, edge_index)
        return output

class SAGE_RES(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()

        self.relu = torch.nn.ReLU()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, out_channels)
        self.res = torch.nn.Linear(in_channels, out_channels)
        # self.norm1 = BatchNorm(hidden_channels, 2)
        # self.norm2 = BatchNorm(hidden_channels, 2)

    def forward(self, node_feature, edge_index):

        output1 = self.relu(self.conv1(node_feature, edge_index))
        # output1 = self.norm1(output1)
        output2 = self.relu(self.conv2(output1, edge_index))
        # output2 = self.norm2(output2)
        output3 = self.conv3(output2, edge_index)
        output_res = self.res(node_feature)
        return (output3 + output_res) * 0.5

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()

        self.relu = torch.nn.ReLU()
        self.conv1 = GATConv(in_channels, hidden_channels, add_self_loops=False)
        self.conv2 = GATConv(hidden_channels, hidden_channels, add_self_loops=False)
        self.conv3 = GATConv(hidden_channels, out_channels, add_self_loops=False)

    def forward(self, node_feature, edge_index):

        output1 = self.relu(self.conv1(node_feature, edge_index))
        output2 = self.relu(self.conv2(output1, edge_index))
        output = self.conv3(output2, edge_index)
        return output

class GAT_RES(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()

        self.relu = torch.nn.ReLU()
        self.conv1 = GATConv(in_channels, hidden_channels, add_self_loops=False)
        self.conv2 = GATConv(hidden_channels, hidden_channels, add_self_loops=False)
        self.conv3 = GATConv(hidden_channels, out_channels, add_self_loops=False)
        self.res = torch.nn.Linear(in_channels, out_channels)

    def forward(self, node_feature, edge_index):

        output1 = self.relu(self.conv1(node_feature, edge_index))
        output2 = self.relu(self.conv2(output1, edge_index))
        output3 = self.conv3(output2, edge_index)
        output_res = self.res(node_feature)
        return (output3 + output_res) * 0.5

class Embedder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        # [0]: movie, feature dim: 18
        # [1]: user, feature dim: 24
        movies = movielens_raw[node_types[0]]
        users = movielens_raw[node_types[1]]
        self.sage = SAGE(in_channels, hidden_channels, out_channels)
        self.sage = to_hetero(self.sage, metadata=movielens_raw.metadata())
        self.sage_res = SAGE_RES(in_channels, hidden_channels, out_channels)
        self.sage_res = to_hetero(self.sage_res, metadata=movielens_raw.metadata())
        self.gat = GAT(in_channels, hidden_channels, out_channels)
        self.gat = to_hetero(self.gat, metadata=movielens_raw.metadata())
        self.gat_res = GAT_RES(in_channels, hidden_channels, out_channels)
        self.gat_res = to_hetero(self.gat_res, metadata=movielens_raw.metadata())
        self.linear_movie = torch.nn.Linear(movies.num_node_features, in_channels)
        self.linear_user = torch.nn.Linear(users.num_node_features, in_channels)
        self.gnn = self.sage_res

    def forward(self, hetero_data):
        features = {
            node_types[0]: self.linear_movie(hetero_data[node_types[0]].x),
            node_types[1]: self.linear_user(hetero_data[node_types[1]].x)
        }
        embeddings = self.gnn(features, hetero_data.edge_index_dict)
        return embeddings

In [15]:
def calc_emb_similarity(node_embs, edge_index, method="cosine"):
    # node_types[1] = "user"
    # node_types[0] = "movie"
    if method == "cosine":
        return torch.sum(node_embs[node_types[1]][edge_index[0]] * node_embs[node_types[0]][edge_index[1]], 1)

In [16]:
from torch_geometric.loader import LinkNeighborLoader
relation_rate = edge_types[0]
train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[40, 20, 10],
    neg_sampling="binary",
    neg_sampling_ratio=1.0,
    edge_label_index=(relation_rate, train_data[relation_rate].edge_label_index),
    edge_label=train_data[relation_rate].edge_label,
    batch_size=256,
    shuffle=True
)

/usr/local/lib/python3.10/dist-packages/torch_geometric/sampler/neighbor_sampler.py:60: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


In [17]:
def get_loss(scores, labels,loss_fn,edge_weights,mode):

    alpha=0.75
    gamma=5
    loss = loss_fn(scores, labels.float())

    p_t = torch.exp(-loss)
    alpha_tensor = (1 - alpha) + labels * (2 * alpha - 1)
    # alpha_tensor = torch.where(labels == -1, 1 - alpha, alpha)
    # print("alpha",alpha_tensor.shape)
    # print("p_t",p_t.shape)
    # print("loss",loss.shape)
    # print("edge",edge_weights.shape)
    if mode == 0:
      f_loss = alpha_tensor * (1 - p_t) ** gamma * loss*edge_weights
    elif mode == 1:
      f_loss = alpha_tensor * (1 - p_t) ** gamma * loss+edge_weights*loss
    elif mode == 2:
      f_loss = alpha_tensor * (1 - p_t) ** gamma * loss
    elif mode == 3:
      f_loss = edge_weights*loss
    elif mode ==4:
      f_loss = loss

    return f_loss.mean()

In [18]:
from tqdm import tqdm
from torch_geometric.utils import negative_sampling

def train(model, dataloader, optimizer, loss_fn,user_importance,mode):
    correct_count = 0
    all_count = 0
    loss = 0
    model.train()
    for batch in tqdm(dataloader):
        optimizer.zero_grad()

        node_embeddings = model(batch)
        labels = batch[edge_types[0]].edge_label
        similarities = calc_emb_similarity(node_embeddings, batch[edge_types[0]].edge_label_index)

        predictions = similarities.sigmoid() > 0.5
        correct_count += torch.sum(predictions == labels)
        all_count += len(labels)

        edge_weights = edge_importance(user_importance,batch[edge_types[0]].edge_label_index,labels)
        loss = get_loss(similarities, labels,loss_fn,edge_weights,mode)

        loss.backward()
        optimizer.step()
    return model, (float(correct_count) / float(all_count))


In [19]:
from sklearn.metrics import roc_auc_score

@torch.no_grad()
def test(model, hetero_data):
    model.eval()
    node_embs = model(hetero_data)
    node_embs = calc_emb_similarity(node_embs, hetero_data[edge_types[0]].edge_label_index).view(-1).sigmoid()
    return roc_auc_score(hetero_data[edge_types[0]].edge_label.cpu().numpy(), node_embs.cpu().numpy())

In [20]:
# import pandas as pd
# import matplotlib.pyplot as plt

In [21]:
# columns = ['Training Acc', 'Validation Acc', 'Test Acc']
# results_df = pd.DataFrame(columns=columns)

In [22]:
epochs = 30

val_acc = []
test_acc = []
for i in range(3):
    best_val_auc = final_test_auc = 0
    model = Embedder(movielens_raw[node_types[0]].num_node_features, hidden_channels=128, out_channels=64)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-2)
    loss_fn = torch.nn.BCEWithLogitsLoss(reduction='none')
    for epoch in range(1, epochs + 1):

      model, acc = train(model, train_loader, optimizer, loss_fn,user_score,0)
      val_auc = test(model, val_data)
      test_auc = test(model, test_data)
      if val_auc > best_val_auc:
          best_val_auc = val_auc
      final_test_auc = test_auc
      print(f'Epoch: {epoch:03d}, Training Accuracy: {acc:.4f}, Val AUC: {val_auc:.4f}, Test AUC: {test_auc:.4f}')
      # results_df = results_df.append({'Training Acc': acc,
      #                                 'Validation Acc': val_auc,
      #                                 'Test Acc': test_auc},
      #                                ignore_index=True)
    val_acc.append(best_val_auc)
    test_acc.append(final_test_auc)



100%|██████████| 54/54 [00:05<00:00,  9.92it/s]


Epoch: 001, Training Accuracy: 0.5354, Val AUC: 0.7042, Test AUC: 0.6954


100%|██████████| 54/54 [00:06<00:00,  8.86it/s]


Epoch: 002, Training Accuracy: 0.6334, Val AUC: 0.7877, Test AUC: 0.7797


100%|██████████| 54/54 [00:05<00:00,  9.43it/s]


...

Epoch: 028, Training Accuracy: 0.7866, Val AUC: 0.8953, Test AUC: 0.8984


100%|██████████| 54/54 [00:05<00:00, 10.70it/s]


Epoch: 029, Training Accuracy: 0.7858, Val AUC: 0.8878, Test AUC: 0.8871


100%|██████████| 54/54 [00:05<00:00, 10.67it/s]


Epoch: 030, Training Accuracy: 0.7877, Val AUC: 0.8924, Test AUC: 0.8944


In [23]:
def calculate_standard_error(tensor):
    mean = torch.mean(tensor)
    std_dev = torch.std(tensor, unbiased=True)
    n = torch.numel(tensor)
    standard_error = std_dev / torch.sqrt(torch.tensor(n, dtype=torch.float))

    return standard_error

In [24]:
print(f'Mean Validation AUC: {torch.mean(torch.tensor(val_acc)):.4f} +/- {calculate_standard_error(torch.tensor(val_acc)):.4f}')
print(f'Mean Test AUC: {torch.mean(torch.tensor(test_acc)):.4f} +/- {calculate_standard_error(torch.tensor(test_acc)):.4f}')

Mean Validation AUC: 0.9002 +/- 0.0018
Mean Test AUC: 0.8959 +/- 0.0027
